In [1]:
!pip install selenium
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time
import re

In [2]:
from selenium import webdriver

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

In [4]:
path = r'.\chromedriver.exe'

In [5]:
dbfile = sqlite3.connect(r'C:\Users\User\AmazonBooks2.db')

In [6]:
#Create cursor to interact with database
cur = dbfile.cursor()

#Create table to put on database
cur.execute("CREATE TABLE Books (Id, Book_Content, Title, Author, Publisher, Publ_Date, Length, Price, Number_Reviews, Avg_Rating, Amazon_Rank);")
    
dbfile.commit()

In [7]:
#only sword and sorcery fantasy books
urlAllBooks = """
https://www.amazon.com/s?i=stripbooks&bbn=16190&rh=n%3A283155%2Cn%3A1000%2Cn%3A25%2Cn%3A16190%2Cn%3A16205%2Cp_n_feature_browse-bin%3A618073011%2Cp_20%3AEnglish&s=date-desc-rank&dc&page={my_page}&Adv-Srch-Books-Submit.x=35&Adv-Srch-Books-Submit.y=15&field-datemod=12&field-dateop=During&field-dateyear=2017&qid=1583268097&rnid=16190&unfiltered=1&ref=sr_pg_2"""


### GETTING THE RIGHT URL:

#### 1. AMAZON

#### 2. BOOKS

#### 3. ADVANCED SEARCH

#### 4. SUBJECT: SCIENCE FICTION AND FANTASY / FORMAT: KINDLE EDITION / LANGUAGE: ENGLISH / PUBLICATION DATE: DURING (MONTH) (YEAR) / SORT RESULTS BY: PUBLICATION DATE

#### 5. SELECT SUBGENRE: FANTASY / SELECT SUBGENRE: SWORD AND SORCERY (PRECAUTION: SELECT PUBLICATION ON THE DROP DOWN MENU TOP RIGHT)

### REMEMBER TO:

#### CHANGE THE URL (ALSO ADD {my_page})

#### CHANG THE ID IN CNTING_BOOKS

#### CHANGE THE NUMBER OF PAGES IN THE RANGE OF i

In [8]:
%%time

#Scrapping pages for all content we want and saving to database
cnting_books = 2406

#REMEMBER TO SUM ONE!
number_of_pages = 18

driver = webdriver.Chrome(path, options = options)

for i in range(1, number_of_pages):
    driver.get(urlAllBooks.format(my_page = i))
    time.sleep(4)
    
    for j in range(0, 16):
        
        foundBook = True
        try:
            driver.find_element_by_xpath("""//div[@data-index = "{}"]//span[@data-component-type = "s-product-image"]/*""".format(j)).click()
            time.sleep(4)
        except:
            print("Couldn't find book in position {} on page {}".format(j, i))
            foundBook = False
        
        if foundBook:
            
            my_dict = {"Id": None, "Book_Content": None, "Title": None, "Author": None, "Publisher": None, "Publ_Date": None, "Length": None, "Price": None, "Number_Reviews": None, "Avg_Rating": None, "Amazon_Rank": None}
            
            my_dict["Id"] = cnting_books
            cnting_books += 1
            
            loaded_page = driver.page_source
            parsed_page = BeautifulSoup(loaded_page, 'html.parser')
            
            try:
                my_dict["Title"] = " ".join(filter(None, str(parsed_page.find('span', {"id": "ebooksProductTitle"}).text.replace("\n", "")).split(" ")))
            except:
                print("Couldn't fetch Title for book on position {} in page {}".format(j, i))
            
            try:
                all_authors = parsed_page.find_all('span', class_ = "author")
                authors = ""
            
                for m in range(0, len(all_authors)):
                    one_author = str(all_authors[m].find('a', {"class": "a-link-normal contributorNameID"}).text.replace("\n", "").replace(",", ""))
                    if "Visit Amazon's" in one_author:
                        one_author.replace("Visit Amazon's ", "")
                        re.sub("Page", "", one_author, 1)
                    authors = authors + one_author + (" ") + (all_authors[m].find('span', {"class": "contribution"}).text.replace("\n", "").replace(",", ""))
                my_dict["Author"] = authors
            except:
                try:
                    all_authors = parsed_page.find_all('span', class_ = "author")
                    authors = ""
            
                    for m in range(0, len(all_authors)):
                        one_author = str(all_authors[m].find('a', {"class": "a-link-normal"}).text.replace("\n", "").replace(",", ""))
                        if "Visit Amazon's" in one_author:
                            one_author.replace("Visit Amazon's ", "")
                            re.sub("Page", "", one_author, 1)
                        authors = authors + one_author + (" ") + (all_authors[m].find('span', {"class": "contribution"}).text.replace("\n", "").replace(",", ""))
                    my_dict["Author"] = authors
                except:
                    print("Couldn't fetch Author for book on position {} in page {}".format(j, i))
            
            try:
                table_details = parsed_page.find('table', {"id": "productDetailsTable"}).find_all("li")
                for element in table_details:
                    try:
                        if 'Publisher:' in element.text:
                            my_dict["Publisher"] = str(element.text[:element.text.find("(")].replace("Publisher:", "").replace("; 1 edition", ""))
                    except:
                        pass
                        #print("Couldn't fetch Publisher for book on position {} in page {}".format(j, i))
                    try:
                        if 'Publication Date:' in element.text:
                            my_dict["Publ_Date"] = str(element.text.replace("Publication Date: ", ""))
                    except:
                        pass
                        #print("Couldn't fetch the Publication Date for book on position {} in page {}".format(j, i))
                    try:
                        if 'Print Length:' in element.text:
                            my_dict["Length"] = int(element.text.replace("Print Length:", "").replace("pages", ""))
                    except:
                        pass
                        #print("Couldn't fetch Book Length for book on position {} in page {}".format(j, i))
                    try:
                        if 'Amazon Best Sellers Rank:' in element.text:
                            my_dict["Amazon_Rank"] = int(element.text[:element.text.find("in Kindle Store")].replace(",","").replace("Amazon Best Sellers Rank:", "").replace("#", "").replace("Paid", "").replace("Free", "").replace("\n", ""))
                    except:
                        print("Couldn't fetch Amazon Rank for book on position {} in page {}".format(j, i))
            except:
                print("Couldn't fetch Table Details for book on position {} in page {}".format(j, i))
            
            try:
                my_dict["Price"] = float(parsed_page.find('span', {"class": "a-size-base a-color-price a-color-price"}).text.replace("$", ""))
            except:
                print("Couldn't fetch Price for book on position {} in page {}".format(j, i))
            
            try:
                my_dict["Number_Reviews"] = int(parsed_page.find('span', {"id": "acrCustomerReviewText"}).text.replace("ratings", "").replace("rating", "").replace("\n", "").replace(",", ""))
            except:
                pass
                #print("Couldn't fetch Number of Reviews for book on position {} in page {}".format(j, i))
            
            try:
                my_dict["Avg_Rating"] = float(parsed_page.find_all('span', {'class': "a-icon-alt"})[0].text.replace("out of 5 stars", "").replace("\n", ""))
            except:
                pass
                #print("Couldn't fetch Average Review value for book on position {} in page {}".format(j, i))
            
            clickText = True
            try: 
                driver.find_element_by_id('ebooks-img-canvas').click()
            except:
                print("Couldn't click to open book in position {} on page {}".format(j, i))
                clickText = False
                    
            time.sleep(4)
            
            if clickText:
                has_iframe = True
                try:
                    driver.switch_to.frame("sitbReaderFrame")
                except:
                    has_iframe = False
    
                loaded_book = driver.page_source
                parsed_content = BeautifulSoup(loaded_book, 'html.parser')

                if has_iframe:
                    #When there's an iframe
                    try:
                        book = parsed_content.find_all('p')
                        if book == []:
                            print("Book on position {} in page {} is empty".format(j, i))
                        fix_book = ""
                        for element in book:
                            fix_book = fix_book + str(element.text) + "<p>"
                        my_dict["Book_Content"] = fix_book 
                    except:
                        print("Couldn't parse html for book on position {} in page {}".format(j, i))
                        
                else:
                    #When there's no iframe
                    try:
                        book = parsed_content.find('div', {"id": "sitbReaderKindleSample"}).find_all(lambda x: x.name == 'div' and x.find_all('div') == [])
                        if book == []:
                            print("Book on position {} in page {} is empty".format(j, i))
                        fix_book = ""
                        for element in book:
                            fix_book = fix_book + str(element.text) + "<p>"
                        my_dict["Book_Content"] = fix_book 
                    except:
                        print("Couldn't parse html for book on position {} in page {}".format(j, i))
            
            temp = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in my_dict.items()]))
            temp.to_sql('Books', dbfile, if_exists = 'append', index = False)
        
        driver.get(urlAllBooks.format(my_page = i))
    dbfile.commit()

Couldn't parse html for book on position 9 in page 7
Couldn't fetch Price for book on position 12 in page 13
Couldn't parse html for book on position 12 in page 13
Couldn't fetch Title for book on position 1 in page 14
Couldn't fetch Table Details for book on position 1 in page 14
Couldn't fetch Price for book on position 1 in page 14
Couldn't click to open book in position 1 on page 14
Couldn't find book in position 2 on page 14


KeyboardInterrupt: 

In [9]:
#Scrapping pages for unfetched/unprocessed books
cnting_books = 1013

unfetched = [(11, 11), (11, 12), (11, 13), (11, 14), (11, 15), (14, 0), (14, 1), (14, 2), (14, 3), (14, 4), (14, 5), (14, 6), (14, 7), (13 ,0), (13, 1), (13, 2), (13, 3), (13, 4), (13, 5), (13, 6), (13, 7), (13, 8), (13, 9), (13, 10), (13, 12), (13, 13), (13, 14), (13, 15)]

driver = webdriver.Chrome(path, options = options)

for unfetch in unfetched:
    driver.get(urlAllBooks.format(my_page = unfetch[0]))
    time.sleep(4)
        
    foundBook = True
    try:
        driver.find_element_by_xpath("""//div[@data-index = "{}"]//span[@data-component-type = "s-product-image"]/*""".format(unfetch[1])).click()
        time.sleep(4)
    except:
        print("Couldn't find book in position {} on page {}".format(unfetch[1], unfetch[0]))
        foundBook = False
        
    if foundBook:
            
        my_dict = {"Id": None, "Book_Content": None, "Title": None, "Author": None, "Publisher": None, "Publ_Date": None, "Length": None, "Price": None, "Number_Reviews": None, "Avg_Rating": None, "Amazon_Rank": None}
            
        my_dict["Id"] = cnting_books
        cnting_books += 1
            
        loaded_page = driver.page_source
        parsed_page = BeautifulSoup(loaded_page, 'html.parser')
            
        try:
            my_dict["Title"] = " ".join(filter(None, str(parsed_page.find('span', {"id": "ebooksProductTitle"}).text.replace("\n", "")).split(" ")))
        except:
            print("Couldn't fetch Title for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        try:
            all_authors = parsed_page.find_all('span', class_ = "author")
            authors = ""
            
            for m in range(0, len(all_authors)):
                one_author = str(all_authors[m].find('a', {"class": "a-link-normal contributorNameID"}).text.replace("\n", "").replace(",", ""))
                if "Visit Amazon's" in one_author:
                    one_author.replace("Visit Amazon's ", "")
                    re.sub("Page", "", one_author, 1)
                authors = authors + one_author + (" ") + (all_authors[m].find('span', {"class": "contribution"}).text.replace("\n", "").replace(",", ""))
            my_dict["Author"] = authors
        except:
            try:
                all_authors = parsed_page.find_all('span', class_ = "author")
                authors = ""
            
                for m in range(0, len(all_authors)):
                    one_author = str(all_authors[m].find('a', {"class": "a-link-normal"}).text.replace("\n", "").replace(",", ""))
                    if "Visit Amazon's" in one_author:
                        one_author.replace("Visit Amazon's ", "")
                        re.sub("Page", "", one_author, 1)
                    authors = authors + one_author + (" ") + (all_authors[m].find('span', {"class": "contribution"}).text.replace("\n", "").replace(",", ""))
                my_dict["Author"] = authors
            except:
                print("Couldn't fetch Author for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        try:
            table_details = parsed_page.find('table', {"id": "productDetailsTable"}).find_all("li")
            for element in table_details:
                try:
                    if 'Publisher:' in element.text:
                        my_dict["Publisher"] = str(element.text[:element.text.find("(")].replace("Publisher:", "").replace("; 1 edition", ""))
                except:
                    pass
                    #print("Couldn't fetch Publisher for book on position {} in page {}".format(unfetch[1], unfetch[0]))
                try:
                    if 'Publication Date:' in element.text:
                        my_dict["Publ_Date"] = str(element.text.replace("Publication Date: ", ""))
                except:
                    pass
                    #print("Couldn't fetch the Publication Date for book on position {} in page {}".format(unfetch[1], unfetch[0]))
                try:
                    if 'Print Length:' in element.text:
                        my_dict["Length"] = int(element.text.replace("Print Length:", "").replace("pages", ""))
                except:
                    pass
                    #print("Couldn't fetch Book Length for book on position {} in page {}".format(unfetch[1], unfetch[0]))
                try:
                    if 'Amazon Best Sellers Rank:' in element.text:
                        my_dict["Amazon_Rank"] = int(element.text[:element.text.find("in Kindle Store")].replace(",","").replace("Amazon Best Sellers Rank:", "").replace("#", "").replace("Paid", "").replace("Free", "").replace("\n", ""))
                except:
                    print("Couldn't fetch Amazon Rank for book on position {} in page {}".format(unfetch[1], unfetch[0]))
        except:
            print("Couldn't fetch Table Details for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        try:
            my_dict["Price"] = float(parsed_page.find('span', {"class": "a-size-base a-color-price a-color-price"}).text.replace("$", ""))
        except:
            print("Couldn't fetch Price for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        try:
            my_dict["Number_Reviews"] = int(parsed_page.find('span', {"id": "acrCustomerReviewText"}).text.replace("ratings", "").replace("rating", "").replace("\n", "").replace(",", ""))
        except:
            pass
            #print("Couldn't fetch Number of Reviews for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        try:
            my_dict["Avg_Rating"] = float(parsed_page.find_all('span', {'class': "a-icon-alt"})[0].text.replace("out of 5 stars", "").replace("\n", ""))
        except:
            pass
            #print("Couldn't fetch Average Review value for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        clickText = True
        try: 
            driver.find_element_by_id('ebooks-img-canvas').click()
        except:
            print("Couldn't click to open book in position {} on page {}".format(unfetch[1], unfetch[0]))
            clickText = False
                    
        time.sleep(4)
            
        if clickText:
            has_iframe = True
            try:
                driver.switch_to.frame("sitbReaderFrame")
            except:
                has_iframe = False
    
            loaded_book = driver.page_source
            parsed_content = BeautifulSoup(loaded_book, 'html.parser')

            if has_iframe:
                #When there's an iframe
                try:
                    book = parsed_content.find_all('p')
                    if book == []:
                         print("Book on position {} in page {} is empty".format(unfetch[1], unfetch[0]))
                    fix_book = ""
                    for element in book:
                          fix_book = fix_book + str(element.text) + "<p>"
                    my_dict["Book_Content"] = fix_book 
                except:
                    print("Couldn't parse html for book on position {} in page {}".format(unfetch[1], unfetch[0]))
                        
            else:
                #When there's no iframe
                try:
                    book = parsed_content.find('div', {"id": "sitbReaderKindleSample"}).find_all(lambda x: x.name == 'div' and x.find_all('div') == [])
                    if book == []:
                        print("Book on position {} in page {} is empty".format(unfetch[1], unfetch[0]))
                    fix_book = ""
                    for element in book:
                        fix_book = fix_book + str(element.text) + "<p>"
                    my_dict["Book_Content"] = fix_book 
                except:
                    print("Couldn't parse html for book on position {} in page {}".format(unfetch[1], unfetch[0]))
            
        temp = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in my_dict.items()]))
        temp.to_sql('Books', dbfile, if_exists = 'append', index = False)
        
    driver.get(urlAllBooks.format(my_page = unfetch[0]))
dbfile.commit()

In [ ]:
#If something went wrong, delete it

#CAREFUL, DON'T DO THIS IF ALL IS OKAY
cur = dbfile.cursor()

delete = """
    DELETE FROM Books
    
    WHERE Id >= 663 Id <= 879
"""

#cur.execute(delete)
dbfile.commit()

In [ ]:
dbfile.close()